# Sporulation

In [1]:
from __future__ import print_function, division, absolute_import

import sys

import qminospy
from qminospy.me2 import ME_NLP

# python imports
from copy import copy
import re
from os.path import join, dirname, abspath
import sys
from collections import defaultdict
import pickle

# third party imports
import pandas as pd
import cobra
from tqdm import tqdm
import numpy as np
import scipy

# COBRAme
import cobrame
from cobrame.util import building, mu, me_model_interface
from cobrame.io.json import save_json_me_model, save_reduced_json_me_model

# ECOLIme
import bacillusme
from bacillusme import (transcription, translation, flat_files, generics, formulas, compartments)
from cobrame.util.helper_functions import *

import copy
from scipy import stats
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
ecoli_files = dirname(abspath(bacillusme.__file__))
pd.set_option('display.max_colwidth', None)

/home/jt/work/cobrapy-0.5.11/cobra/io/sbml3.py:24: UserWarning: Install lxml for faster SBML I/O
  warn("Install lxml for faster SBML I/O")
/home/jt/work/cobrapy-0.5.11/cobra/io/__init__.py:12: UserWarning: cobra.io.sbml requires libsbml
  warn("cobra.io.sbml requires libsbml")


/home/jt/work/cobrapy-0.5.11/cobra/__init__.py
/home/jt/work/cobrame/cobrame/__init__.py
/home/jt/work/sporeme/bacillusme/__init__.py


In [2]:
def add_transport_to_model(model,mets):
    for met in mets:
        rxn = cobrame.MEReaction('{}_transport'.format(met))
        model.add_reactions([rxn])
        rxn.add_metabolites({
            '{}_c'.format(met):-1,
            '{}_s'.format(met):1
        })
        rxn.bounds = (-1000,1000)
        print(rxn.reaction)

### Essentiality of FS genes

In [3]:
import pickle
with open("./sporeme_solution_v4_proteomics_spoAQ.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [4]:
add_transport_to_model(sporeme,['gtp','ctp','utp','gdp','cdp','udp','adp','atp'])

gtp_c <=> gtp_s
ctp_c <=> ctp_s
utp_c <=> utp_s
gdp_c <=> gdp_s
cdp_c <=> cdp_s
udp_c <=> udp_s
adp_c <=> adp_s
atp_c <=> atp_s


In [5]:
from bacillusme.analysis import sensitivity as ss

In [6]:
genes = ['BSU' + g.id.split('RNA_BSU')[1] for g in sporeme.metabolites.query(re.compile('RNA_BSU.*_s$'))]

In [7]:
solve_me_model(sporeme,
                   max_mu = 0.1,
                   min_mu = .01,
                   using_soplex=False,
                   precision = 1e-6,
                   growth_key = 'sigma',
                   verbosity=2)

iter	muopt    	a     	b     	mu1       	stat1
Finished compiling expressions in 52.720371 seconds
Finished substituting S,lb,ub in 1.770504 seconds
Finished makeME_LP in 0.255897 seconds
Getting MINOS parameters from ME_NLP...
1 0.0 0.0 0.05 0.05 1
Finished substituting S,lb,ub in 1.800112 seconds
Finished makeME_LP in 0.254141 seconds
Getting MINOS parameters from ME_NLP...
2 0.025 0.025 0.05 0.025 optimal
Finished substituting S,lb,ub in 1.798148 seconds
Finished makeME_LP in 0.256972 seconds
Getting MINOS parameters from ME_NLP...
3 0.025 0.025 0.037500000000000006 0.037500000000000006 1
Finished substituting S,lb,ub in 1.789847 seconds
Finished makeME_LP in 0.257528 seconds
Getting MINOS parameters from ME_NLP...
4 0.03125 0.03125 0.037500000000000006 0.03125 optimal
Finished substituting S,lb,ub in 1.791263 seconds
Finished makeME_LP in 0.257550 seconds
Getting MINOS parameters from ME_NLP...
5 0.034375 0.034375 0.037500000000000006 0.034375 optimal
Finished substituting S,lb,ub i

In [ ]:
number_per_step = 100
for i in range(0,int(len(genes)/number_per_step)+1):
    a,b = (i*number_per_step,i*number_per_step + number_per_step)
    b = min(b,len(genes))
    print(i+1,a,b)
    flux_responses = ss.all_flux_responses(sporeme,genes[a:b],NP=10,precision=1e-6, solution=sporeme.solution,
                       single_change_function='gene_knockout',growth_key='sigma')
    flux_responses.to_csv('essentiality_raw_results_FS_spoAQ_{}.csv'.format(i+1))

1 0 100


dummy_reaction_FWD_SPONT_s response (10 threads):   0%|          | 0/100 [00:00<?, ?it/s]

 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
3FFF0000000000000000000000000000
 3FF0000000000000
3FFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set option Print frequency to       100000
 Calling miopti to set option Scale option to            2
 Calling miopti to set option Iteration limit to      2000000
 Calling miopti to set option Expand frequency to       100000
 46293E5939A08CEA
406293E5939A08CEA000000000000000
 Calling mioptr to set option Penalty parameter to    100.0000

dummy_reaction_FWD_SPONT_s response (10 threads):   1%|          | 1/100 [21:30<35:28:40, 1290.10s/it]

  
 Quad MINOS finished.
 inform =           0
 ninf   =           0
 sinf   =   0.00000000000000000000000000000000000      
 obj    =   2.20896931684631852690728581443354229E-0004
BFF2CF41231DAC6349E08280C6BFB43E
 BF2CF41231DAC635
BFFF0000000000000000000000000000
 BFF0000000000000
3FFF0000000000000000000000000000
 3FF0000000000000
 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
3FFF0000000000000000000000000000
 3FF0000000000000
3FFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set op

...

  
 Quad MINOS finished.
 inform =           1
 ninf   =         122
 sinf   =   1.57175432865242445307782349884400044E-0003
 obj    =   0.00000000000000000000000000000000000      
                               0
                0
                               0
                0
80000000000000000000000000000000
 8000000000000000
  
 Quad MINOS finished.
 inform =           1
 ninf   =           2
 sinf   =   1.43518046710522203490756413034909191E-0005
 obj    =   0.00000000000000000000000000000000000      
                               0
                0
                               0
                0
80000000000000000000000000000000
 8000000000000000
  
 Quad MINOS finished.
 inform =           1
 ninf   =           1
 sinf   =   8.51716788879569610615418577045156110E-0008
 obj    =   0.00000000000000000000000000000000000      
                               0
                0
                               0
                0
80000000000000000000000000000000
 800000000000000

dummy_reaction_FWD_SPONT_s response (10 threads):  27%|██▋       | 27/100 [2:31:47<6:54:59, 341.08s/it]

  
 Quad MINOS finished.
 inform =           0
 ninf   =           0
 sinf   =   0.00000000000000000000000000000000000      
 obj    =   2.20896931684631852690617237684003892E-0004
BFF2CF41231DAC6349E07E3214C837B1
 BF2CF41231DAC635
BFFF0000000000000000000000000000
 BFF0000000000000
3FFF0000000000000000000000000000
 3FF0000000000000
 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
3FFF0000000000000000000000000000
 3FF0000000000000
3FFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set op

dummy_reaction_FWD_SPONT_s response (10 threads):  28%|██▊       | 28/100 [2:33:13<5:17:23, 264.49s/it]

 Allocating name1, name2
 Allocating pi, rc, xn
 Allocated pi, rc, xn
 No Jacobian matrix since no nonlinear obj or constraints
3FFF0000000000000000000000000000
 3FF0000000000000
3FFF0000000000000000000000000000
C083D6329F1C35CA5000000000000000
 C83D6329F1C35CA5
C083D6329F1C35CA5000000000000000
4083D6329F1C35CA5000000000000000
 483D6329F1C35CA5
4083D6329F1C35CA5000000000000000
 Calling miopt to set option: Maximize
 Calling miopt to set option: Solution No
 Calling miopti to set option New basis file to           11
 Calling miopti to set option Save frequency to       500000
 Calling miopti to set option Print level to            0
 Calling miopti to set option Print frequency to       100000
 Calling miopti to set option Scale option to            2
 Calling miopti to set option Iteration limit to      2000000
 Calling miopti to set option Expand frequency to       100000
 46293E5939A08CEA
406293E5939A08CEA000000000000000
 Calling mioptr to set option Penalty parameter to    100.0000

In [ ]:
import os

In [ ]:
flux_responses = {}
f = 'essentiality_raw_results_FS_spoAQ'
for i in os.listdir():
    if f not in i: continue
    if f+'.csv' == i: continue
    print(i)
    d = pd.read_csv(i,index_col=0).to_dict()
    for k,v in d.items():
        flux_responses[k] = v
flux_responses = pd.DataFrame.from_dict(flux_responses)
flux_responses.to_csv('{}.csv'.format(f))

### Essentiality of MC genes

In [28]:
with open("./sporeme_solution_v4_proteomics_spoAQ.pickle", "rb") as outfile:
    sporeme = pickle.load(outfile)

In [29]:
add_transport_to_model(sporeme,['gtp','ctp','utp','gdp','cdp','udp','adp','atp'])

gtp_c <=> gtp_s
ctp_c <=> ctp_s
utp_c <=> utp_s
gdp_c <=> gdp_s
cdp_c <=> cdp_s
udp_c <=> udp_s
adp_c <=> adp_s
atp_c <=> atp_s


In [ ]:
solve_me_model(sporeme,
                   max_mu = 0.1,
                   min_mu = .01,
                   using_soplex=False,
                   precision = 1e-6,
                   growth_key = 'sigma',
                   verbosity=2)

In [30]:
from bacillusme.analysis import sensitivity as ss

In [31]:
genes = ['BSU' + g.id.split('RNA_BSU')[1] for \
         g in sporeme.metabolites.query(re.compile('RNA_BSU.*[^s]$'))]

In [ ]:
for i in range(0,int(len(genes)/100)+1):
    a,b = (i*100,i*100 + 100)
    b = min(b,len(genes))
    print(i+1,a,b)
    flux_responses = ss.all_flux_responses(sporeme,genes[a:b],NP=10,precision=1e-6, solution=sporeme.solution,
                       single_change_function='gene_knockout',growth_key='sigma')
    flux_responses.to_csv('essentiality_raw_results_MC_spoAQ_{}.csv'.format(i+1))

In [ ]:
import os
flux_responses = {}
f = 'essentiality_raw_results_MC_spoAQ'
for i in os.listdir():
    if f not in i: continue
    if f+'.csv' == i: continue
    print(i)
    d = pd.read_csv(i,index_col=0).to_dict()
    print(len(d))
    for k,v in d.items():
        flux_responses[k] = v
flux_responses = pd.DataFrame.from_dict(flux_responses)
flux_responses.to_csv('{}.csv'.format(f))